# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 819.2 kB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [4]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [7]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "qwen/qwen3-30b-a3b:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [8]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745957654-INauGZlJC8kDFWbbYvPE', 'provider': 'Chutes', 'model': 'qwen/qwen3-30b-a3b:free', 'object': 'chat.completion', 'created': 1745957654, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The question of the meaning of life is one of the most profound and enduring inquiries in human history, and it has no single, universally accepted answer. Different perspectives—philosophical, religious, scientific, and personal—offer varied interpretations. Here are some key approaches to consider:\n\n### 1. **Philosophical Perspectives**  \n   - **Existentialism**: Thinkers like Jean-Paul Sartre and Albert Camus argue that life has no inherent meaning, and it is up to individuals to create their own purpose through choices, actions, and commitments.  \n   - **Absurdism**: Camus suggests that while life may be inherently meaningless, we can find freedom and authenticity by embracing the "

In [9]:
data = response.json()
data['choices'][0]['message']['content']

'The question of the meaning of life is one of the most profound and enduring inquiries in human history, and it has no single, universally accepted answer. Different perspectives—philosophical, religious, scientific, and personal—offer varied interpretations. Here are some key approaches to consider:\n\n### 1. **Philosophical Perspectives**  \n   - **Existentialism**: Thinkers like Jean-Paul Sartre and Albert Camus argue that life has no inherent meaning, and it is up to individuals to create their own purpose through choices, actions, and commitments.  \n   - **Absurdism**: Camus suggests that while life may be inherently meaningless, we can find freedom and authenticity by embracing the "absurd" (the clash between humanity\'s search for meaning and the universe\'s silence).  \n   - **Stoicism**: Focuses on living virtuously, cultivating inner tranquility, and aligning with nature or reason.  \n   - **Nihilism**: Proposes that life has no intrinsic meaning, though some interpretation

## 2. Using OpenAI Agents SDK

In [10]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

Function calls itself,  
Base case prevents endless loops—  
Solves big problems small.


# OpenRouter 404 Error Solution

## Error - No endpoints found matching
```python
NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy. Enable prompt training here: https://openrouter.ai/settings/privacy', 'code': 404}}
```

## Cause
This error occurs when OpenRouter API can't find endpoints matching your data policy, typically because prompt training is disabled.

## Solution

1. **Enable Prompt Training**:
   - Visit [OpenRouter Privacy Settings](https://openrouter.ai/settings/privacy)
   - Toggle ON "Prompt Training" option

2. **Re-run your code** after enabling

![OpenRouter Settings Screenshot](./openrouter.png)
*(Example: Enable prompt training in privacy settings)*

## Prevention
Keep prompt training enabled for uninterrupted API access.
```